# Visualize meadows on Google Satellite

In [ ]:
import os, ee, pickle
import warnings, math
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta

In [ ]:
import geemap
import ipyleaflet

In [ ]:
ee.Initialize()

In [ ]:
shapefile = gpd.read_file("files/AllPossibleMeadows_2024-02-12.shp")

In [ ]:
landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate('2018-10-01', '2019-10-01')

In [ ]:
def maskImage(image):
    qa = image.select('QA_PIXEL')
    # mask out cloud based on bits in QA_pixel
    dilated_cloud = qa.bitwiseAnd(1 << 1).eq(0)
    cirrus = qa.bitwiseAnd(1 << 2).eq(0)
    cloud = qa.bitwiseAnd(1 << 3).eq(0)
    cloudShadow = qa.bitwiseAnd(1 << 4).eq(0)
    cloud_mask = dilated_cloud.And(cirrus).And(cloud).And(cloudShadow)
    return image.updateMask(cloud_mask)

In [ ]:
meadowId = 17238
feature = shapefile.loc[meadowId, 'geometry']
tot_area = shapefile.loc[meadowId, 'Area_km2']
if feature.geom_type == 'Polygon':
    shapefile_bbox = ee.Geometry.Polygon(list(feature.exterior.coords)).buffer(-5)
elif feature.geom_type == 'MultiPolygon':
    shapefile_bbox = ee.Geometry.MultiPolygon(list(list(poly.exterior.coords) for poly in feature.geoms)).buffer(-5)

In [ ]:
landsat_images = landsat8_collection.filterBounds(shapefile_bbox).map(maskImage)
landsat_image = landsat_images.first().clip(shapefile_bbox)

In [ ]:
band_values = landsat_image.reduceRegion(ee.Reducer.toList(), shapefile_bbox, 30).getInfo()
len(band_values['SR_B4'])

In [ ]:
def zoom_level(area=tot_area):
    ''' zoom_level ranges from 10 (largest of 369.83082 km2) to 19 (smallest of 0.0007 km2) for all polygons
     Each zoom-out approximately quadruples the area viewed (hence log 2)
     calculate deviation or zoom-out extent from 19 '''
    tradeoff = math.log2(area/0.0007)
    return (19 - round(tradeoff/2))

In [ ]:
Map = geemap.Map(center=list(feature.centroid.coords[0])[::-1], zoom=zoom_level())
gdf_selected = gpd.GeoDataFrame(geometry=[feature])
geo_data = ipyleaflet.GeoData(geo_dataframe=gdf_selected, style={'color': 'red', 'fillOpacity':0.01})

Map.addLayer(landsat_image.clip(shapefile_bbox).mask(1), {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3}, 'Landsat Image', True, 0.5)
Map.add_layer(geo_data)

display(Map)

# This is for visualizing Geotiffs of AGB, BGB and GHG predictions

In [ ]:
from osgeo import gdal
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
import rasterio
from rasterio.plot import show

filename = 'files/Image_meadow_17238_GHG.tif'
with rasterio.open(filename) as src:
    band = src.read(1)
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
    plt.figure(figsize=(10, 10))
    plt.imshow(band, extent=extent, cmap='viridis')
    plt.title(filename.split('.tif')[0][-3:] + ' Band')
    plt.colorbar(label='Prediction')
    plt.xlabel('Longitude', fontsize=10)
    plt.ylabel('Latitude', fontsize=10)
    plt.xticks(rotation='vertical', ha='center')
    plt.show()

In [ ]:
coords = shapefile_bbox.bounds().coordinates().getInfo()[0]
xmin, ymin = coords[0]
xmax, ymax = coords[2]
num_subregions = round(math.sqrt(len(band_values['SR_B4'])/1250))

subregion_width = (xmax - xmin) / num_subregions
subregion_height = (ymax - ymin) / num_subregions
subregions = []

In [ ]:
for i in range(num_subregions):
    for j in range(num_subregions):
        subregion = ee.Geometry.Rectangle([xmin + i*subregion_width, ymin + j*subregion_height,
                                           xmin + (i+1)*subregion_width, ymin + (j+1)*subregion_height])
        subregions.append(subregion)

In [ ]:
Map = geemap.Map(center=list(feature.centroid.coords[0])[::-1], zoom=zoom_level())
gdf_selected = gpd.GeoDataFrame(geometry=[feature])
geo_data = ipyleaflet.GeoData(geo_dataframe=gdf_selected, style={'color': 'red', 'fillOpacity':0.01})

Map.addLayer(landsat_image.clip(subregions[1]).mask(1), {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3}, 'Landsat Image', True, 0.5)
Map.add_layer(geo_data)

display(Map)

In [ ]:
len(subregions)